# Analysis for Data Augmentation

In [2]:
import pandas as pd
from ast import literal_eval

In [3]:
train_df = pd.read_csv('train/train.csv')
test_df = pd.read_csv('test/test_data.csv')

In [4]:
# subject, object 분리
train_df.columns
train_df['subject_entity'] = train_df['subject_entity'].apply(lambda x: literal_eval(x))
train_df['object_entity'] = train_df['object_entity'].apply(lambda x: literal_eval(x))

train_df['sub_word'] = train_df['subject_entity'].apply(lambda x: x['word'])
train_df['obj_word'] = train_df['object_entity'].apply(lambda x: x['word'])

train_df['sub_start_idx'] = train_df['subject_entity'].apply(lambda x: x['start_idx'])
train_df['sub_end_idx'] = train_df['subject_entity'].apply(lambda x: x['end_idx'])

train_df['obj_start_idx'] = train_df['object_entity'].apply(lambda x: x['start_idx'])
train_df['obj_end_idx'] = train_df['object_entity'].apply(lambda x: x['end_idx'])

In [5]:
train_df.sample(3)

,id,sentence,subject_entity,object_entity,label,source,sub_word,obj_word,sub_start_idx,sub_end_idx,obj_start_idx,obj_end_idx
13753,13753,그녀는 전립선암으로 인해 출마를 포기한 루돌프 줄리아니 대신 공화당의 릭 라지오와 ...,"{'word': '루돌프 줄리아니', 'start_idx': 22, 'end_idx...","{'word': '공화당', 'start_idx': 34, 'end_idx': 36...",no_relation,wikipedia,루돌프 줄리아니,공화당,22,29,34,36
13348,13348,그는 “미국 시장 수요 감소에도 불구하고 현대차의 판매는 증가하고 있다. 특히 제네...,"{'word': '현대차', 'start_idx': 23, 'end_idx': 25...","{'word': '제네시스', 'start_idx': 44, 'end_idx': 4...",org:product,wikitree,현대차,제네시스,23,25,44,47
19794,19794,김용범 기획재정부 1차관은 이날 파행을 겪은 지 이틀 만에 농민단체와 간담회를 다시...,"{'word': '기획재정부', 'start_idx': 4, 'end_idx': 8...","{'word': '김용범', 'start_idx': 0, 'end_idx': 2, ...",org:top_members/employees,wikitree,기획재정부,김용범,4,8,0,2


In [6]:
a = '안녕 나는 (테스트) 문장이야.'
print(a[12:14])
print(list(a)[12:14])

result = list(a)
result.insert(12, '*')
result.insert(14+1, '*')
print(result)
print(''.join(result))

문장
['문', '장']
['안', '녕', ' ', '나', '는', ' ', '(', '테', '스', '트', ')', ' ', '*', '문', '장', '*', '이', '야', '.']
안녕 나는 (테스트) *문장*이야.


In [7]:
# @가 문장에 없으므로 이것을 entity marker로 쓰자.
train_df['sentence'].str.contains('@').sum()

0

In [14]:
train_df['sentence'].str.contains('#').sum()

0

In [8]:
train_df['sentence'].loc[0]

'〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.'

In [9]:
# 문장에 entity를 직접 표시
train_df['sentence_with_entity_marker'] = None
train_df['sub_extracted'] = None
train_df['obj_extracted'] = None 

for i in train_df.index:
    
    sub_start_idx = train_df['sub_start_idx'].loc[i]
    sub_end_idx = train_df['sub_end_idx'].loc[i]
    
    obj_start_idx = train_df['obj_start_idx'].loc[i]
    obj_end_idx = train_df['obj_end_idx'].loc[i]
    
    train_df['sub_extracted'].loc[i] = train_df['sentence'].loc[i][sub_start_idx:sub_end_idx+1]
    train_df['obj_extracted'].loc[i] = train_df['sentence'].loc[i][obj_start_idx:obj_end_idx+1]
    
    
    train_df['sentence_with_entity_marker'].loc[i] = list(train_df['sentence'].loc[i])
    
    # sub_entity, obj_entity 중 앞에 있는 엔티티에 먼저 마커를 붙여주면, 뒤에 있는 엔티티는 인덱스가 2만큼 밀려난다.  
    if sub_start_idx > obj_start_idx:
    
        train_df['sentence_with_entity_marker'].loc[i].insert(sub_start_idx, '@')
        train_df['sentence_with_entity_marker'].loc[i].insert(sub_end_idx+2, '@')

        train_df['sentence_with_entity_marker'].loc[i].insert(obj_start_idx, '@')
        train_df['sentence_with_entity_marker'].loc[i].insert(obj_end_idx+2, '@')

    else:
        train_df['sentence_with_entity_marker'].loc[i].insert(obj_start_idx, '@')
        train_df['sentence_with_entity_marker'].loc[i].insert(obj_end_idx+2, '@')
        
        train_df['sentence_with_entity_marker'].loc[i].insert(sub_start_idx, '@')
        train_df['sentence_with_entity_marker'].loc[i].insert(sub_end_idx+2, '@')

    train_df['sentence_with_entity_marker'].loc[i] = ''.join(train_df['sentence_with_entity_marker'].loc[i])
    
# 결과를 보면, 

# 균일가 생활용품점 (주)@아성다이소@(대@표 박@정부)는 코로나19 바이러스로 어려...	
# 와 같이, 인덱스는 맞는데도 마커 삽입이 잘 안 된 경우가 있다.

train_df

/opt/conda/envs/boost/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,id,sentence,subject_entity,object_entity,label,source,sub_word,obj_word,sub_start_idx,sub_end_idx,obj_start_idx,obj_end_idx,sentence_with_entity_marker,sub_extracted,obj_extracted
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia,비틀즈,조지 해리슨,24,26,13,18,〈Something〉는 @조지 해리슨@이 쓰고 @비틀즈@가 1969년 앨범 《Abb...,비틀즈,조지 해리슨
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree,민주평화당,대안신당,19,23,14,17,호남이 기반인 바른미래당·@대안신당@·@민주평화당@이 우여곡절 끝에 합당해 민생당(...,민주평화당,대안신당
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree,광주FC,한국프로축구연맹,21,24,34,41,K리그2에서 성적 1위를 달리고 있는 @광주FC@는 지난 26일 @한국프로축구연맹@...,광주FC,한국프로축구연맹
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree,아성다이소,박정부,13,17,22,24,균일가 생활용품점 (주)@아성다이소@(대표 @박정부@)는 코로나19 바이러스로 어려...,아성다이소,박정부
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia,요미우리 자이언츠,1967,22,30,0,3,@1967@년 프로 야구 드래프트 1순위로 @요미우리 자이언츠@에게 입단하면서 등번...,요미우리 자이언츠,1967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree,유기준,부산 서구·동구,93,95,100,107,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,유기준,부산 서구·동구
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia,최시형,손병희,7,9,17,19,"법포는 다시 @최시형@, 서병학, @손병희@ 직계인 북접과 다시 서장옥, 전봉준, ...",최시형,손병희
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree,완도군,신우철,0,2,7,9,@완도군@(군수 @신우철@)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사...,완도군,신우철
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia,JTBC,중앙홀딩스,6,9,21,25,"중앙일보, @JTBC@ 회장을 지낸 이후 @중앙홀딩스@ 회장, 재단법인 한반도평화만...",JTBC,중앙홀딩스


In [10]:
for i in train_df['sentence_with_entity_marker'][0:10]:
    print('sentence: ', i, '\n')

sentence:  〈Something〉는 @조지 해리슨@이 쓰고 @비틀즈@가 1969년 앨범 《Abbey Road》에 담은 노래다. 

sentence:  호남이 기반인 바른미래당·@대안신당@·@민주평화당@이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다. 

sentence:  K리그2에서 성적 1위를 달리고 있는 @광주FC@는 지난 26일 @한국프로축구연맹@으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다. 

sentence:  균일가 생활용품점 (주)@아성다이소@(대표 @박정부@)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다. 

sentence:  @1967@년 프로 야구 드래프트 1순위로 @요미우리 자이언츠@에게 입단하면서 등번호는 8번으로 배정되었다. 

sentence:  : 유엔, 유럽 의회, @북대서양 조약 기구@ (@NATO@), 국제이주기구, 세계 보건 기구 (WHO), 지중해 연합, 이슬람 협력 기구, 유럽 안보 협력 기구, 국제 통화 기금, 세계무역기구 그리고 프랑코포니. 

sentence:  그에 따라 나폴리와 계약을 연장한 마라도나는 @1989@년 팀을 UEFA컵 정상으로 인도했으며 이듬해에는 유럽 챔피언 @AC 밀란@을 상대로 승리를 거두고 다시 한 번 세리에A에서 정상에 등극했다. 

sentence:  @박용오@(朴容旿, @1937년 4월 29일@(음력 3월 19일)(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임했다. 

sentence:  중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 @타이베이@로 수도를 옮기는 것을 결정해, 남아있는 @중화민국@군의 병력이나 국가, 개인의 재산등을 속속 타이완으로 옮기기 시작해, 12월에는 중앙 정부 기구도 모두 이전해 타이베이 시를 중화민국의 새로운 수도로 삼았다. 

sentence:  특히 

In [11]:
# 형태소 분석
from konlpy.tag import Mecab
mecab = Mecab()

In [12]:
for i in train_df['sentence_with_entity_marker'][0:10]:
    print('sentence: ', mecab.pos(i), '\n')

sentence:  [('〈', 'SY'), ('Something', 'SL'), ('〉', 'SY'), ('는', 'JX'), ('@', 'SY'), ('조지', 'NNP'), ('해리슨', 'NNP'), ('@', 'SY'), ('이', 'JKS'), ('쓰', 'VV'), ('고', 'EC'), ('@', 'SY'), ('비틀즈', 'NNP'), ('@', 'SY'), ('가', 'JKS'), ('1969', 'SN'), ('년', 'NNBC'), ('앨범', 'NNG'), ('《', 'SY'), ('Abbey', 'SL'), ('Road', 'SL'), ('》', 'SY'), ('에', 'JKB'), ('담', 'VV'), ('은', 'ETM'), ('노래', 'NNG'), ('다', 'VCP+EF'), ('.', 'SF')] 

sentence:  [('호남', 'NNG'), ('이', 'JKS'), ('기반', 'NNG'), ('인', 'VCP+ETM'), ('바른', 'VA+ETM'), ('미래', 'NNG'), ('당', 'XSN'), ('·', 'SC'), ('@', 'SY'), ('대안', 'NNG'), ('신', 'XPN'), ('당', 'NNG'), ('@·@', 'SY'), ('민주', 'NNG'), ('평화', 'NNG'), ('당', 'XSN'), ('@', 'SY'), ('이', 'MM'), ('우여곡절', 'NNG'), ('끝', 'NNG'), ('에', 'JKB'), ('합당', 'XR'), ('해', 'XSA+EC'), ('민생', 'NNG'), ('당', 'XSN'), ('(', 'SSO'), ('가칭', 'NNG'), (')', 'SSC'), ('으로', 'JKB'), ('재', 'XPN'), ('탄생', 'NNG'), ('한다', 'XSV+EF'), ('.', 'SF')] 

sentence:  [('K', 'SL'), ('리그', 'NNG'), ('2', 'SN'), ('에서', 'JKB'), ('성적', 'NNG'),

In [13]:
# 반말 -> 존댓말 테스트
import os

from tqdm.auto import tqdm
from AIpjt.utils import Utils
from AIpjt.changer import Changer

model = Changer()

for i in train_df['sentence_with_entity_marker'][0:10]:
    print('sentence: ', model.changer(i), '\n')

sentence:  Something는 @조지 해리슨@이 쓰고 @비틀즈@가 1969년 앨범 Abbey Road에 담으신 노래입니다. 

sentence:  호남이 기반인 바른미래당·@대안신당@·@민주평화당@이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생합니다. 

sentence:  K리그2에서 성적 1위를 달리고 있는 @광주FC@는 지난 26일 @한국프로축구연맹@으로부터 관중 유치 성과와 마케팅 성과를 인정받어 풀 스타디움상과 플러스 스타디움상을 수상하었습니다. 

sentence:  균일가 생활용품점 (주)@아성다이소@(대표 @박정부@)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달하었다고 10일 밝히었습니다. 

sentence:  @1967@년 프로 야구 드래프트 1순위로 @요미우리 자이언츠@에게 입단하면서 등번호는 8번으로 배정되었습니다. 

sentence:  : 유엔, 유럽 의회, @북대서양 조약 기구@ (@NATO@), 국제이주기구, 세계 보건 기구 (WHO), 지중해 연합, 이슬람 협력 기구, 유럽 안보 협력 기구, 국제 통화 기금, 세계무역기구 그리고 프랑코포니. 

sentence:  그에 따라 나폴리와 계약을 연장한 마라도나는 @1989@년 팀을 UEFA컵 정상으로 인도하었으며 이듬해에는 유럽 챔피언 @AC 밀라인@을 상대로 승리를 거두고 다시 한 번 세리에A에서 정상에 등극하었습니다. 

sentence:  @박용오@(, @1937년 4월 29일@(음력 3월 19일)(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임하었습니다. 

sentence:  중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 @타이베이@로 수도를 옮기는 것을 결정하어, 남어있는 @중화민국@군의 병력이나 국가, 개인의 재산등을 속속 타이완으로 옮기기 시작하어, 12월에는 중앙 정부 기구도 모두 이전해 타이베이 시를 중화민국의 새롭으신 수도로 삼었습니다. 


## 제한적인 EDA

In [ ]:
### 문장 안에 부사가 있을 때 random swqp, random deletion수행

In [22]:
import random

def adverb_detector(sentence):
    # Tokenizing    
    tokenized = mecab.pos(sentence)
    
    # 어절 정보도 활용하기 위해 (원문 공백 기준 split, tokenized)와 같은 튜플 형태로 저장한다.
    eojeol_token_tuple = (sentence, tokenized)
    
    for i, token in enumerate(tokenized):
        try:
            if tokenized[i-1] != '@' and tokenized[i+1] != '@' and token[1] == "MAG":
                return True
        except:
            pass

def 

In [23]:
adverb_detector('밥을 굉장히 많이 먹었다.')

True

In [24]:
train_df['sentence_with_entity_marker'].apply(lambda x: adverb_detector(x)).count()

9558

In [25]:
mecab.pos('나는 밥을 상당하게 많이 먹었다.')

[('나', 'NP'),
 ('는', 'JX'),
 ('밥', 'NNG'),
 ('을', 'JKO'),
 ('상당', 'NNG'),
 ('하', 'XSA'),
 ('게', 'EC'),
 ('많이', 'MAG'),
 ('먹', 'VV'),
 ('었', 'EP'),
 ('다', 'EF'),
 ('.', 'SF')]

In [29]:
mecab.morphs('나는 밥을 상당하게 많이 먹었다.')

['나', '는', '밥', '을', '상당', '하', '게', '많이', '먹', '었', '다', '.']